# Resolving human object recognition: A time-frequency analysis

Vani, Ben, & Jeanne

NEUR182: Machine Learning w/ Neural Signal

Prof. Michael Spezio

Fall 2020

In [4]:
import os.path as op
import numpy as np
import numpy.matlib
from pandas import read_csv
import matplotlib.pyplot as plt
import gc # for memory cleaning

from sklearn import preprocessing, model_selection, linear_model, metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.manifold import MDS
from sklearn.preprocessing import scale as CenterScale
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

import scipy
from scipy import stats

import mne
from mne.io import read_raw_fif, concatenate_raws
from mne.datasets import visual_92_categories

[function definitions]

In [5]:
# performs frequency decomposition
# returns power as np.ndarray

def fre_decomp(fftMEG, MorletFamily, nEvents, nConvolution, nShift):
    fftGW = np.fft.fft(MorletFamily[:], nConvolution)
    fftconv = fftMEG * np.matlib.repmat(fftGW, nEvents, 1)
    conv_result = np.fft.ifft(fftconv, nConvolution, 1)
    conv_result = conv_result[:, (nShift):(conv_result.shape[1] - nShift)]
    power = np.power(np.absolute(conv_result), 2)
    
    del fftGW 
    del fftconv
    del conv_result
    gc.collect()
    
    return power


In [6]:
# implements Matlab's dsearchn()
# returns indices of nearest points as np.ndarray

def dsearchn(x, y):
    dist = np.zeros((y.shape[0]), int)
    
    for line in range(0, y.shape[0]):
        distances = np.abs(x - y[line])
        distances.argmin()
        dist[line] = distances.argmin().astype(int)
    
    return dist


In [7]:
# implements Matlab's normpdf()
# returns pdf as float

def normpdf(x, mu=0, sigma=1):
    u = float((x-mu) / abs(sigma))
    y = np.exp(-u*u/2) / (np.sqrt(2*np.pi) * abs(sigma))
    
    return y


In [8]:
# performs feature engineering
# returns feature set as np.ndarray

def FeatureBins(MEG_chan, timeIndx, freqIndx, nEvents):
    nTBins = len(timeIndx)
    nFBins = len(freqIndx)
    featureset = np.zeros((nEvents, nFBins-1, nTBins-1))
   
    for time in range (0, nTBins-1):
        for freq in range (0, nFBins-1):
            temp = MEG_chan[(freqIndx[freq]):(freqIndx[freq + 1]), 
                            (timeIndx[time]):(timeIndx[time + 1]),:]
            featureset[:, freq, time] = stats.zscore(np.median(np.squeeze(np.median(temp,0)),0))
     
    return featureset


In [9]:
# tunes regression models using validation proportion of training set
# returns 

def TuningModels(models, X, Y, val_size, iterations = 100):
    import numpy as np
    from sklearn.model_selection import train_test_split
    results = {}
    for i in models:
        metric_train = []
        metric_val = []
        for j in range(iterations):
            X_train, X_val, y_train, y_val = train_test_split(X,Y,test_size = val_size)
            metric_val.append(metrics.mean_squared_error(y_val,
                                                     models[i].fit(X_train,
                                                                   y_train).predict(X_val)))
            metric_train.append(metrics.mean_squared_error(y_train,
                                                       models[i].fit(X_train, 
                                                                     y_train).predict(X_train)))
        results[i] = [np.mean(metric_train), np.mean(metric_val)]
    return results


[begin data shaping]

In [10]:
data_path = visual_92_categories.data_path()

# define stimulus-trigger mapping (code from King, Leppakangas, & Gramfort)
fname = op.join(data_path, 'visual_stimuli.csv')
conds = read_csv(fname)

In [11]:
# take the first 92 rows
max_trigger = 92
conds = conds[:max_trigger]

In [12]:
conditions = []
for c in conds.values:
    cond_tags = list(c[:2])
    cond_tags += [('not-' if i == 0 else '') + conds.columns[k]
                  for k, i in enumerate(c[2:], 2)]
    conditions.append('/'.join(map(str, cond_tags)))

In [13]:
event_id = dict(zip(conditions, conds.trigger + 1))

In [14]:
nRuns = 1  # 4 for full data (use less to speed up computations)
fname = op.join(data_path, 'sample_subject_%i_tsss_mc.fif')

raws = [read_raw_fif(fname % block, verbose='error')
        for block in range(0, nRuns)]  # ignore filename warnings
raw = concatenate_raws(raws)

events = mne.find_events(raw, min_duration=.002)
events = events[events[:, 2] <= max_trigger]

1374 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93 200 222 244]


In [15]:
picks = mne.pick_types(raw.info, meg=True)
epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None,
                    picks=picks, tmin=-.1, tmax=.4, preload=True)
del raw

Not setting metadata
Not setting metadata
920 matching events found
No baseline correction applied
0 projection items activated
Loading data for 920 events and 501 original time points ...
0 bad epochs dropped


In [16]:
# visual representation of epochs data structure
#epochs.plot(n_epochs=5)

In [17]:
ep50_300 = epochs.copy().crop(0.05, 0.3).get_data()
base10_0 = epochs.copy().crop(-0.1, 0).get_data()

[end data shaping] [begin data processing]

In [18]:
# intialize elements for Morlet wavelets
nWavelets = 236
freq_range = np.array([2,120])
lofreq = freq_range[0]
hifreq = freq_range[1]
freqs = np.linspace(lofreq, hifreq, nWavelets)
Fs = 1000 # Hz, sampling rate
timevec = np.linspace(0.05, 0.3, 251)
timevec_gauss = np.linspace(-2, 2, 4001)
nCycles = 7

MorletFamily = np.empty((0, 4001), float)

# create complex wavelet family
for wavelet in range(0, nWavelets):
        omega = 2 * np.pi * freqs[wavelet];
        sigma = nCycles / omega
        gauss = np.array([normpdf(i,0, sigma)
                    for i in timevec_gauss])
        sig = np.exp(1j * omega * timevec_gauss)
        MorletFamily = np.append(MorletFamily, [sig * gauss], axis = 0)

In [19]:
# initalize elements for transferring time series to frequency domain
nEvents = ep50_300.shape[0]
MEG_Power = np.zeros((nEvents,306,8,25))
nChan = ep50_300.shape[1]
nShift = int((timevec_gauss.size-1)/2)

# time & frequency bins
TimeBins = np.linspace(0.05, 0.3, 26)
FreqBins = np.array([2, 4, 8, 13, 20, 35, 55, 80, 120])
timeIndx = dsearchn(timevec, TimeBins)
freqIndx = dsearchn(freqs, FreqBins)

print("time indices are:") 
print(timeIndx)
print("\nfrequency indices are:")
print(freqIndx)

time indices are:
[  0  10  20  30  40  50  60  70  80  90 100 110 120 130 140 150 160 170
 180 190 200 210 220 230 240 250]

frequency indices are:
[  0   4  12  22  36  66 106 155 235]


In [25]:
# *** loop over channels ***
for chan in range(0, nChan): 

    # process baseline data
    pre_data = np.squeeze(base10_0[:,chan,:])
    pre_nConvolution = timevec_gauss.size + pre_data.shape[1] - 1
    
    # process signal data
    data = np.squeeze(ep50_300[:,chan,:])
    nEvents = data.shape[0]
    nConvolution = timevec_gauss.size + data.shape[1] - 1

    # convert signal & baseline to frequency domain
    pre_fftMEG = np.fft.fft(pre_data[:,:], pre_nConvolution)
    fftMEG = np.fft.fft(data[:,:], nConvolution)

    MEG_chan =  np.empty((0, 251, nEvents))  

    
    # ****** loop over wavelets *******
    for wavelet in range(0, nWavelets):
        pre_power = fre_decomp(pre_fftMEG, MorletFamily[wavelet,:], nEvents, 
                               pre_nConvolution, nShift)
        sig_power = fre_decomp(fftMEG, MorletFamily[wavelet,:], nEvents, 
                               nConvolution, nShift)

        normal_MEG = (np.transpose(sig_power) / 
                      np.matlib.repmat(np.median(np.transpose(pre_power), 0), 251, 1))
        MEG_chan = np.append(MEG_chan, [normal_MEG], axis = 0)
       
    
    # store feature set for current electrode
    MEG_chanBins = FeatureBins(MEG_chan, timeIndx, freqIndx, nEvents)      
    MEG_Power[:,chan,:,:] = MEG_chanBins  # [electrode x trial x wavelet x time]
    
    del MEG_chan
    del MEG_chanBins
    gc.collect()     
    
    print(chan)

0


KeyboardInterrupt: 

[end data processing] [begin data analysis]

In [20]:
# load data features
featureset = np.load('Pt1Featureset.npy')
featureset.shape

(920, 306, 8, 25)

In [21]:
# create labels
y_sup = (epochs.events[:, 2] > 48).astype(int) # set up superordinate classification label
print(y_sup.shape)

classes = set(y_sup) 
print(classes)

(920,)
{0, 1}


In [29]:
# train-test split
Feats_train, Feats_test, OV_train, OV_test = train_test_split(featureset, y_sup, test_size=0.3, random_state=47)

In [30]:
np.save('Feats_train', feats_train)
np.save('Feats_test', feats_test)
np.save('OV_train', OV_train)
np.save('OV_test', OV_test)

In [31]:
Feats_train = np.load('Feats_train.npy')
Feats_test = np.load('Feats_test.npy')
OV_train = np.load('OV_train.npy')
OV_test = np.load('OV_test.npy')

In [33]:
print(Feats_train.shape)
print(Feats_test.shape)

(644, 61200)
(276, 61200)


In [34]:
Feats_train = np.reshape(Feats_train, (644, 61200))
Feats_test = np.reshape(Feats_test, (276, 61200))

In [35]:
print(Feats_train.shape)
print(Feats_test.shape)

(644, 61200)
(276, 61200)


In [7]:
# assess correlation between features to check for multicollinearity
# PVsCorrMat = np.corrcoef(feats_train, rowvar = False)
# PVsCoDMat = np.power(PVsCorrMat, 2)

In [8]:
# print(PVsCodDMat.shape)
# print(PVsCodDMat)

In [ ]:
#import matplotlib.cm as cm
#surf(PVsCoDMat,cm.jet)

In [ ]:
#PCA
#from sklearn.decomposition import PCA
#pca = PCA(n_components = 2)
#X2D = pca.fit_transform(X)

[regularized regression]

In [ ]:
# fit lasso & ridge models with a tuning approach over hyperparameter α (λ)

lasso_params = {'alpha':np.linspace(1e-5,1e-2,30)}
ridge_params = {'alpha':np.linspace(1e-10,20,50)}

models = {'Lasso': GridSearchCV(linear_model.Lasso(tol = 1e-3, random_state=47), 
                                param_grid=lasso_params).fit(Feats_train, OV_train).best_estimator_,
          'Ridge': GridSearchCV(linear_model.Ridge(tol = 1e-3, random_state = 47),
                                 param_grid=ridge_params).fit(Feats_train, OV_train).best_estimator_}

reg_result = TuningModels(models, Feats_train, OV_train, val_size = 0.2, iterations = 47)

In [30]:
# print results
print(reg_result)

# print optimal parameters after tuning
print('Optimal alpha for LASSO is ' + str(models['Lasso'].alpha))
print('Optimal alpha for Ridge is ' + str(models['Ridge'].alpha))

{'Lasso': [0.012464100328162519, 0.3018793766747705], 'Ridge': [7.098829730860406e-08, 0.2628597931900838]}
Optimal alpha for LASSO is 0.01
Optimal alpha for Ridge is 20.0


In [ ]:
# regularized regression -- lasso
LassoModel = Lasso(tol = 1e-3, random_state = 47, alpha = models['Lasso'].alpha)
LassoModel.fit(Feats_train, OV_train)
LassoPredictTest = LassoModel.predict(Feats_test)

# calculate performance metrics
LassoRSqError = 1 - metrics.explained_variance_score(OV_test, LassoPredictTest)
print('Lasso Rsq Error = ' + str(LassoRSqError))
Lasso_b_acc = metrics.balanced_accuracy_score(OV_test, LassoPredictTest)
print('Lasso balanced accuracy = ' + str(Lasso_b_acc))
Lasso_roc_auc = metrics.roc_auc_score(OV_test, LassoPredictTest)
print('Lasso ROC/AUC = ' + str(Lasso_roc_auc))
Lasso_f1 = metrics.f1_score(OV_test, LassoPredictTest)
print('Lasso F1 = ' + str(Lasso_f1)
Lasso_precision = metrics.precision_score(OV_test, LassoPredictTest)
print('Lasso precision = ' + str(Lasso_precision)
Lasso_recall = metrics.recall_score(OV_test, LassoPredictTest)
print('Lasso recall = ' + str(Lasso_recall)
Lasso_confused = confusion_matrix(OV_test, LassoPredictTest)

In [32]:
# regularized regression -- ridge
RidgeModel = Ridge(tol = 1e-3, random_state = 47, alpha = models['Ridge'].alpha)
RidgeModel.fit(Feats_train, OV_train)
RidgePredictTest = RidgeModel.predict(Feats_test)

# calculate performace metrics
RidgeRSqError = 1 - metrics.explained_variance_score(OV_test, RidgePredictTest)
print('Ridge Rsq Error = ' + str(RidgeRSqError))
Ridge_b_acc = metrics.balanced_accuracy_score(OV_test, RidgePredictTest)
print('Ridge balanced accuracy = ' + str(Ridge_b_acc))
Ridge_roc_auc = metrics.roc_auc_score(OV_test, RidgePredictTest)
print('Ridge ROC/AUC = ' + str(Ridge_roc_auc))
Ridge_f1 = metrics.f1_score(OV_test, RidgePredictTest)
print('Ridge F1 = ' + str(Ridge_f1)
Ridge_precision = metrics.precision_score(OV_test, RidgePredictTest)
print('Ridge precision = ' + str(Ridgeo_precision)
Ridge_recall = metrics.recall_score(OV_test, RidgePredictTest)
print('Ridge recall = ' + str(Ridge_recall)
Ridge_confused = confusion_matrix(OV_test, RidgePredictTest)

Lasso Rsq Error = 1.1323441531983698
Ridge Rsq Error = 1.1733581292235875


[SVM]

In [9]:
# svm tuning
def svc_tune(X, y, nfolds, random_state=10):
    tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-2, 1e-3, 1e-4, 1e-5],
                     'C': [ 0.1, 10, 100, 1000]}
                    #{'kernel': ['sigmoid'], 'gamma': [1e-5, 1e-6, 1e-7, 1e-8],
                     #'C': [0.001, 0.10, 0.1, 10, 25, 50, 100, 1000]}
                    #{'kernel': ['linear'], 'C': [0.001, 0.10, 0.1, 10, 25, 50, 100, 1000]}
                   ]
    grid_search = GridSearchCV(SVC(C=1), tuned_parameters, scoring='roc_auc', cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

In [ ]:
best_params = svc_tune(CenterScale(Feats_train), OV_train, 5)

In [25]:
# Train the classifier
# Specify parameters
kernel_choice = 'sigmoid'

# Specify our object
MySVC = SVC(kernel = kernel_choice, C = 10, gamma =0.0005)

# Train our SVC
MySVC.fit(CenterScale(feats_train),OV_train)

SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.0005, kernel='sigmoid',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [27]:
TestPred = MySVC.predict(CenterScale(feats_test))
AUCResult = metrics.roc_auc_score(OV_test,TestPred)
print('AUC = ' + str(AUCResult))
BalAccResult = metrics.balanced_accuracy_score(OV_test,TestPred)
print('Balanced Accuracy = ' + str(BalAccResult))
F1Result = metrics.f1_score(OV_test,TestPred)
print('F1 = ' + str(F1Result))

AUC = 0.5112964527027027
Balanced Accuracy = 0.5112964527027027
F1 = 0.46692607003891046


In [28]:
# Train the classifier
# Specify parameters
kernel_choice = 'linear'

# Specify our object
MyLinearSVC = SVC(kernel = kernel_choice, C = 10)

# Train our SVC
MyLinearSVC.fit(CenterScale(feats_train),OV_train)

SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [29]:
TestPred = MyLinearSVC.predict(CenterScale(feats_test))
AUCResult = metrics.roc_auc_score(OV_test,TestPred)
print('AUC = ' + str(AUCResult))
BalAccResult = metrics.balanced_accuracy_score(OV_test,TestPred)
print('Balanced Accuracy = ' + str(BalAccResult))
F1Result = metrics.f1_score(OV_test,TestPred)
print('F1 = ' + str(F1Result))

AUC = 0.5203758445945946
Balanced Accuracy = 0.5203758445945946
F1 = 0.4924242424242424


[RVM]

In [30]:
from skrvm import RVC
# Specify parameters
kernel_choice = 'linear'

# Specify our object
MyRVC = RVC(kernel = kernel_choice, verbose=False)

# Train our RVC
from sklearn.preprocessing import scale as CenterScale
MyRVC.fit(CenterScale(feats_train),OV_train)

C:\Users\ussha\anaconda3\envs\Neuro182_tf2\lib\site-packages\skrvm\rvm.py:239: RuntimeWarning: divide by zero encountered in log
  np.sum(np.log(1-y[t == 0]), 0))


RVC(alpha=1e-06, beta=1e-06, beta_fixed=False, bias_used=True, coef0=0.0,
    coef1=None, degree=3, kernel='linear', n_iter=3000, n_iter_posterior=50,
    threshold_alpha=1000000000.0, tol=0.001, verbose=False)

In [33]:
# Explore our classifier
print('Number of Relevance Vectors = ' + str(MyRVC.relevance_.shape[0]))
print('Median of Feature Weights (only available for Linear Kernel) = ' + str(np.median(MyRVC.relevance_)))
print('Maximum of Feature Weights (only available for Linear Kernel) = ' + str(np.max(MyRVC.relevance_)))

Number of Relevance Vectors = 105
Median of Feature Weights (only available for Linear Kernel) = -0.19561504771603905
Maximum of Feature Weights (only available for Linear Kernel) = 25.30679633714632


In [32]:
# Test the classifier
from sklearn import metrics
TestPred = MyRVC.predict(CenterScale(feats_test))
AUCResult = metrics.roc_auc_score(OV_test,TestPred)
print('AUC = ' + str(AUCResult))
BalAccResult = metrics.balanced_accuracy_score(OV_test,TestPred)
print('Balanced Accuracy = ' + str(BalAccResult))
F1Result = metrics.f1_score(OV_test,TestPred)
print('F1 = ' + str(F1Result))

AUC = 0.5008445945945946
Balanced Accuracy = 0.5008445945945946
F1 = 0.48327137546468396


[random forest]

In [ ]:
#RF

ncores = 2


# Grid of hyperparameters

# Number of trees in random forest
n_estimators = [int(x) for x in numpy.linspace(start = 20, stop = 2000, num = 50)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in numpy.linspace(10, 110, num=11)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]


# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split}
    

forest = RandomForestClassifier(n_estimators = 100, criterion = 'gini', max_depth = 20,  min_samples_split = 10, max_features = 'sqrt')
forest.fit(Feats_train, OV_train)

In [ ]:
# Grid Search (This is run elsewhere. See the next cell for results)
grid_search = GridSearchCV(estimator = forest, param_grid = random_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(Feats_train, OV_train)
grid_search.best_params_

{'max_depth': 20,
 'max_features': 'sqrt',
 'min_samples_split': 10,
 'n_estimators': 100}

In [ ]:
# Training Prediction
forest_predict_train = forest.predict(Feats_train)

# Best Parameter Evaluation
params_eval = f1_score(OV_train, forest_prediction_train)

# Testing Prediction 
OV_pred = forest.predict(Feats_test)

# Performance Metrics
b_accuracy = balanced_accuracy_score(OV_test, OV_pred)
print(b_accuracy)
roc_auc = roc_auc_score(OV_test, OV_pred)
print(roc_auc)
f1 = f1_score(OV_test, OV_pred)
print(f1)
precision = precision_score(OV_test, OV_pred)
print(precision)
recall = recall_score(OV_test, OV_pred)
print(recall)
confused = confusion_matrix(OV_test, OV_pred)
print(confused)


Results from Performance metrics:
Balanced Accuracy =.5461359797297297
ROC AUC = 0.5461359797297297
F1 = 0.3195876288659794
Precision = 0.6739130434782609
Recall = 0.20945945945945946